In [1]:
import yfinance as yf
import pandas as pd
from ipywidgets import Output
from sklearn.metrics import precision_score, accuracy_score
from sklearn.metrics import precision_score, accuracy_score
predictors = []

In [2]:
stocks = ["^gspc","avgo"]
data =pd.DataFrame([])
pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
first = True
               
for stock in stocks:
    stk = yf.Ticker(stock)  
    stock_data = stk.history(period='max') 
    stock_data=stock_data.loc["1990-1-1":].copy()
    del stock_data["Dividends"]
    del stock_data["Stock Splits"] 
    del stock_data["Open"]
    del stock_data["High"]
    del stock_data["Low"]
    stock_data["Tomarrow"] = stock_data["Close"].shift(-2)
    stock_data["Target"] = (stock_data["Close"]<stock_data["Tomarrow"]).astype(int)
    
    horizons = [2,5,60,250,1000]
    # predictors = []

    for horizon in horizons:
        rolling_averages = stock_data.rolling(horizon).mean()

        ratio_column = f"{stock}_Close_Avg_{horizon}"
        stock_data[ratio_column] = stock_data["Close"] / rolling_averages["Close"]

        trend_column = f"{stock}_Trend_{horizon}"
        stock_data[trend_column] = stock_data.shift(1).rolling(horizon).sum()["Target"]
        
#         v_ratio_col =f"{stock}_Vol_Avg_{horizon}"
#         stock_data[v_ratio_col] = stock_data["Volume"]/rolling_averages["Volume"]
        
        predictors+= [ratio_column, trend_column]
    
    if  first:
        first = False
    else:
        
        
        
        
        
        print("delete pred name")
        del stock_data['Close']
        del stock_data['Volume']
        del stock_data['Target']
    data = pd.concat([data,stock_data],ignore_index = False,axis=1)
    

In [3]:
data=data.dropna()

In [4]:
import sklearn
from sklearn.ensemble import RandomForestClassifier

train = data.iloc[:-100]
test = data.iloc[-100:]

In [5]:
def backtest(data, model, predictors, start=2500, step=250):
    all_predictions = []
    
    for i in range(start, data.shape[0], step):
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
    
    return pd.concat(all_predictions)

In [6]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >=.7] = 1
    preds[preds <.7] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [7]:
model = RandomForestClassifier(n_estimators=200, min_samples_split=50, random_state=1)

In [8]:
predictions = backtest(data, model, predictors)

In [9]:
predictions["Predictions"].value_counts()

Predictions
0.00    4255
1.00     855
Name: count, dtype: int64

In [10]:
prec = precision_score(predictions["Target"], predictions["Predictions"])
acc = accuracy_score(predictions["Target"], predictions["Predictions"])

In [11]:
print('precision: ',prec)
print('accuracy: ',acc)

precision:  0.7707602339181286
accuracy:  0.5238747553816047


In [12]:
predictions["Target"].value_counts() / predictions.shape[0]

Target
1   0.57
0   0.43
Name: count, dtype: float64

In [13]:
pd.concat([predictions.tail(15),data.tail(15)],axis=1)

,Target,Predictions,Close,Volume,Tomarrow,Target,^gspc_Close_Avg_2,^gspc_Trend_2,^gspc_Close_Avg_5,^gspc_Trend_5,^gspc_Close_Avg_60,^gspc_Trend_60,^gspc_Close_Avg_250,^gspc_Trend_250,^gspc_Close_Avg_1000,^gspc_Trend_1000
Date,,,,,,,,,,,,,,,,
2024-02-14 00:00:00-05:00,1,0.00,5000.62,3845600000,5005.57,1,1.00,1.00,1.00,3.00,1.05,42.00,1.14,148.00,1.24,583.00
2024-02-15 00:00:00-05:00,0,0.00,5029.73,4137970000,4975.51,0,1.00,2.00,1.00,3.00,1.06,42.00,1.15,149.00,1.25,584.00
2024-02-16 00:00:00-05:00,0,0.00,5005.57,3833270000,4981.80,0,1.00,1.00,1.00,2.00,1.05,41.00,1.14,149.00,1.24,584.00
2024-02-20 00:00:00-05:00,1,0.00,4975.51,4034880000,5087.03,1,1.00,0.00,1.00,2.00,1.04,40.00,1.13,149.00,1.24,583.00
2024-02-21 00:00:00-05:00,1,0.00,4981.80,3788390000,5088.80,1,1.00,1.00,1.00,3.00,1.04,40.00,1.13,149.00,1.24,583.00
2024-02-22 00:00:00-05:00,0,0.00,5087.03,4051710000,5069.53,0,1.01,2.00,1.01,3.00,1.06,41.00,1.15,150.00,1.26,583.00
2024-02-23 00:00:00-05:00,0,0.00,5088.80,3672790000,5078.18,0,1.00,1.00,1.01,2.00,1.06,41.00,1.15,150.00,1.26,582.00
2024-02-26 00:00:00-05:00,1,0.00,5069.53,3683930000,5069.76,1,1.00,0.00,1.01,2.00,1.05,40.00,1.15,149.00,1.26,582.00
2024-02-27 00:00:00-05:00,1,0.00,5078.18,3925950000,5096.27,1,1.00,1.00,1.00,3.00,1.05,40.00,1.15,150.00,1.26,583.00


In [14]:
feature_imp=pd.Series( model.feature_importances_).sort_index(ascending=True)

In [15]:
pred=pd.Series(predictors)

In [16]:
print(pd.concat([pred,feature_imp],axis=1))

                      0    1
0     ^gspc_Close_Avg_2 0.16
1         ^gspc_Trend_2 0.15
2     ^gspc_Close_Avg_5 0.17
3         ^gspc_Trend_5 0.03
4    ^gspc_Close_Avg_60 0.10
5        ^gspc_Trend_60 0.05
6   ^gspc_Close_Avg_250 0.10
7       ^gspc_Trend_250 0.07
8  ^gspc_Close_Avg_1000 0.09
9      ^gspc_Trend_1000 0.07
